### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data_df = pd.read_csv(file_to_load)
purchase_data_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [2]:
total_players = len(purchase_data_df["SN"].unique())
total_players

576

## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
num_items = len(purchase_data_df["Item Name"].unique())
avg_pr = purchase_data_df["Price"].mean()
tot_purchases = len(purchase_data_df["Price"])
tot_revenue = purchase_data_df["Price"].sum()

#summary_df = pd.DataFrame( [num_items,avg_pr,tot_purchases,tot_revenue],
#                          columns=["Number of Unique Items",
 #                          "Average Purchase Price",
 #                          "Total Number of Purchases",
 #                          "Total Revenue"])
summary_df = pd.DataFrame({
    "Number of Unique Items": [num_items],
    "Average Purchase Price": [avg_pr],
    "Total Number of Purchases": [tot_purchases],
    "Total Revenue": [tot_revenue]
})
summary_df

,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,179,3.050987,780,2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [156]:

male_df = purchase_data_df.loc[purchase_data_df["Gender"] == "Male"]
female_df = purchase_data_df.loc[purchase_data_df["Gender"] == "Female"]
other_df = purchase_data_df.loc[purchase_data_df["Gender"] == "Other / Non-Disclosed"]

male_count = male_df["SN"].nunique()
female_count = female_df["SN"].nunique()
other_count = other_df["SN"].nunique()
total_players = purchase_data_df["SN"].nunique()
#type(purchase_data_df["Gender"].value_counts())

gender_demo_df = pd.DataFrame({
    " ": ["Male", "Female", "Other"],
    "Count": [male_count, female_count, other_count],
    "Percentage": [male_count/total_players, female_count/total_players, other_count/total_players]
    
})
gender_demo_df.set_index([" "])
gender_demo_df.sort_index()

,,Count,Percentage
0,Male,484,0.840278
1,Female,81,0.140625
2,Other,11,0.019097



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [167]:
gender_df = purchase_data_df.groupby(['Gender'])
gender_pur_count = pd.DataFrame(purchase_data_df['Gender'].value_counts())

gender_avg_pr = gender_df['Price'].mean()
gender_total_value = gender_df['Price'].sum()
#gender_avg_per = gender_pur_count/gender_demo_df["Count"]*gender_avg_pr

purch_df = pd.merge(gender_total_value, gender_avg_pr, on="Gender")
purch_df = purch_df.rename(columns={"Price_x":"Total Purchase Value","Price_y":"Average Purchase Price"})
purch_df['Purchase Count'] = gender_pur_count
#purch_df.sort_values("Total Purchase Value", ascending=False) 
purch_df['Avg Total Purchase per Person'] = purch_df.loc[:,'Total Purchase Value'] / gender_pur_count

purch_df

#Still need avg per person!!!

ValueError: Wrong number of items passed 4, placement implies 1

## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [126]:
bins = [0, 9, 14, 19, 24, 29, 34, 39, 120]
group_names = ['< 10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']
purchase_data_df['Age Group'] = pd.cut(purchase_data_df["Age"], bins, labels=group_names, include_lowest=True)
age_df = purchase_data_df.groupby('Age Group')
age_summary_df = pd.DataFrame({'Total Count': age_df['SN'].nunique(),
                              'Percentage of Players': age_df['SN'].nunique()/total_players})

age_summary_df

,Total Count,Percentage of Players
Age Group,,
< 10,17,0.029514
10-14,22,0.038194
15-19,107,0.185764
20-24,258,0.447917
25-29,77,0.133681
30-34,52,0.090278
35-39,31,0.053819
40+,12,0.020833


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [141]:
#age_purch_count = age_df['Age Group'].value_counts()
#age_purch_count
#age_avg_pr = age_df["Price"].mean()
#age_tot_value = age_df["Price"].sum()
age_purchases_df = pd.DataFrame({'Purchase Count': age_df["Age Group"].count(),
                                 'Average Purchase Price': age_df["Price"].mean(),
                                 'Total Purchase Value': age_df["Price"].sum()
                                })
age_purchases_df
#still need avg per person!!!!

,Purchase Count,Average Purchase Price,Total Purchase Value
Age Group,,,
< 10,23,3.353478,77.13
10-14,28,2.956429,82.78
15-19,136,3.035956,412.89
20-24,365,3.052219,1114.06
25-29,101,2.900990,293.00
30-34,73,2.931507,214.00
35-39,41,3.601707,147.67
40+,13,2.941538,38.24


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [147]:
spenders = purchase_data_df.groupby("SN")
top_spenders_df = pd.DataFrame({'Purchase Count': spenders["Purchase ID"].count(),
                                 'Average Purchase Price': spenders["Price"].mean(),
                                 'Total Purchase Value': spenders["Price"].sum()
                                })
top_spenders_df = top_spenders_df.sort_values("Total Purchase Value", ascending=False)
top_spenders_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,3.792000,18.96
Idastidru52,4,3.862500,15.45
Chamjask73,3,4.610000,13.83
Iral74,4,3.405000,13.62
Iskadarya95,3,4.366667,13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [153]:
items = purchase_data_df.groupby(["Item ID", "Item Name"])
items_df = pd.DataFrame({'Purchase Count': items["Purchase ID"].count(),
                         'Average Purchase Price': items["Price"].mean(),
                         'Total Purchase Value': items["Price"].sum(),
                         })
items_df = items_df.sort_values("Purchase Count", ascending=False)
items_df.head()

,,Purchase Count,Average Purchase Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,4.614615,59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.230000,50.76
145,Fiery Glass Crusader,9,4.580000,41.22
132,Persuasion,9,3.221111,28.99
108,"Extraction, Quickblade Of Trembling Hands",9,3.530000,31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [154]:
items_df = items_df.sort_values("Total Purchase Value", ascending=False)
items_df.head()

,,Purchase Count,Average Purchase Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,4.614615,59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.230000,50.76
82,Nirvana,9,4.900000,44.10
145,Fiery Glass Crusader,9,4.580000,41.22
103,Singed Scalpel,8,4.350000,34.80
